In [100]:
import pandas as pd
import re

In [101]:
mrt_master_df = pd.read_csv('Data/Master Sets/TrainStationMaster.csv')

In [102]:
mrt_master_df.head()

,Unnamed: 0,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,PT_CODE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME
0,0,2024-03,WEEKDAY,11,TRAIN,NS7,21943,15130
1,1,2024-03,WEEKENDS/HOLIDAY,11,TRAIN,NS7,13062,14512
2,2,2024-03,WEEKDAY,16,TRAIN,SW4,2124,3233
3,3,2024-03,WEEKENDS/HOLIDAY,16,TRAIN,SW4,1532,1736
4,4,2024-03,WEEKENDS/HOLIDAY,10,TRAIN,CC5,1886,2659


In [103]:
station_network_list = mrt_master_df['PT_CODE'].unique()
station_network_list.sort()
station_network_list
station_interchanges = [i for i in station_network_list if '/' in i]
station_network_lines = pd.array([re.sub('[0-9]', '', i) for i in station_network_list]).unique()
station_network_lines = [i for i in station_network_lines if '/' not in i]

In [104]:
station_network_list

array(['BP10', 'BP11', 'BP12', 'BP13', 'BP2', 'BP3', 'BP4', 'BP5',
       'BP6/DT1', 'BP7', 'BP8', 'BP9', 'CC10/DT26', 'CC11', 'CC12',
       'CC14', 'CC16', 'CC17/TE9', 'CC19/DT9', 'CC2', 'CC20', 'CC21',
       'CC23', 'CC24', 'CC25', 'CC26', 'CC27', 'CC28', 'CC3', 'CC4/DT15',
       'CC5', 'CC6', 'CC7', 'CC8', 'CE1/DT16', 'CG1/DT35', 'CG2', 'DT10',
       'DT10/TE11', 'DT13', 'DT17', 'DT18', 'DT2', 'DT20', 'DT21', 'DT22',
       'DT23', 'DT24', 'DT25', 'DT27', 'DT28', 'DT29', 'DT3', 'DT30',
       'DT31', 'DT33', 'DT34', 'DT5', 'DT6', 'DT7', 'DT8', 'EW1', 'EW10',
       'EW11', 'EW12/DT14', 'EW14/NS26', 'EW15', 'EW16/NE3/TE17', 'EW17',
       'EW18', 'EW19', 'EW2/DT32', 'EW20', 'EW21/CC22', 'EW22', 'EW23',
       'EW24/NS1', 'EW25', 'EW26', 'EW27', 'EW28', 'EW29', 'EW3', 'EW30',
       'EW31', 'EW32', 'EW33', 'EW4', 'EW5', 'EW6', 'EW7', 'EW8/CC9',
       'EW9', 'NE1/CC29', 'NE10', 'NE11', 'NE12/CC13', 'NE13', 'NE14',
       'NE15', 'NE16/STC', 'NE17/PTC', 'NE4/DT19', 'NE5', 'NE7/DT12

In [117]:
def generate_line_net(line: str):
    if line not in station_network_lines:
        raise ValueError('Not a valid MRT Line')
    count = 1
    adj = {}
    for station in station_network_list:
        if line in station:
            if f"TE{count}" in station_network_list:
                adj[f"{line}{count}"] = {}
                adj[f"{line}{count}"][f"{line}{count}"] = 0
                if f"{line}{count-1}" in station_network_list:
                    adj[f"{line}{count}"][f"{line}{count-1}"] = 1
                    if f"{line}{count-1}" not in adj:
                        adj[f"{line}{count-1}"] = {}
                    adj[f"{line}{count-1}"][f"{line}{count}"] = 1
                if f"{line}{count+1}" in station_network_list:
                    adj[f"{line}{count}"][f"{line}{count+1}"] = 1
                    if f"{line}{count+1}" not in adj:
                        adj[f"{line}{count+1}"] = {}
                    adj[f"{line}{count+1}"][f"{line}{count}"] = 1
            count += 1
    
    return adj

In [120]:
adj = generate_line_net('TE')

In [121]:
# MANUAL ADJUSTMENT FOR INTERCHANGE STATIONS
for station in station_interchanges:
    if 'TE' in station:
        print(station)
        adj[station] = {}

# ONLY 6 Stations for TE Line
adj['CC17/TE9']['TE8'] = 1
adj['TE8']['CC17/TE9'] = 1

adj['CC17/TE9']['DT10/TE11'] = 1
adj['DT10/TE11']['CC17/TE9'] = 1

adj['DT10/TE11']['TE12'] = 1
adj['TE12']['DT10/TE11'] = 1

adj['EW16/NE3/TE17']['TE16'] = 1
adj['TE16']['EW16/NE3/TE17'] = 1
adj['EW16/NE3/TE17']['TE18'] = 1
adj['TE18']['EW16/NE3/TE17'] = 1

adj['NS27/CE2/TE20']['TE19'] = 1
adj['TE19']['NS27/CE2/TE20'] = 1
adj['NS27/CE2/TE20']['TE22'] = 1
adj['TE22']['NS27/CE2/TE20'] = 1

adj['NS27/CE2/TE20']['TE19'] = 1
adj['TE19']['NS27/CE2/TE20'] = 1
adj['NS27/CE2/TE20']['TE22'] = 1
adj['TE22']['NS27/CE2/TE20'] = 1

adj['NS9/TE2']['TE1'] = 1
adj['TE1']['NS9/TE2'] = 1
adj['NS9/TE2']['TE3'] = 1
adj['TE3']['NS9/TE2'] = 1

adj['TE14/NS22']['TE13'] = 1
adj['TE13']['TE14/NS22'] = 1
adj['TE14/NS22']['TE15'] = 1
adj['TE15']['TE14/NS22'] = 1

CC17/TE9
DT10/TE11
EW16/NE3/TE17
NS27/CE2/TE20
NS9/TE2
TE14/NS22


In [122]:
adj_matrix = pd.DataFrame(adj).fillna(0)
adj_matrix

,TE1,TE3,TE4,TE5,TE6,TE7,TE8,TE11,TE12,TE13,...,TE26,TE27,TE28,TE29,CC17/TE9,DT10/TE11,EW16/NE3/TE17,NS27/CE2/TE20,NS9/TE2,TE14/NS22
TE1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
NS9/TE2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TE3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
TE4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TE5,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TE6,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TE7,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TE8,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
CC17/TE9,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
TE11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [114]:
adj_matrix = adj_matrix.sort_index(axis=1).copy()
adj_matrix = adj_matrix.sort_index().copy()
adj_matrix

,CC17/TE9,DT10/TE11,EW16/NE3/TE17,NS27/CE2/TE20,NS9/TE2,TE1,TE11,TE12,TE13,TE14/NS22,...,TE26,TE27,TE28,TE29,TE3,TE4,TE5,TE6,TE7,TE8
CC17/TE9,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
DT10/TE11,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EW16/NE3/TE17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NS27/CE2/TE20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NS9/TE2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
TE1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TE11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TE12,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TE13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TE14/NS22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
adj_matrix.to_csv('Data\Master Sets\TE_Line_Adjacency_Matrix.csv')